## Set up API Key for LLM

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

## Set up Diversity Measure

In [11]:
from langdiversity.measures import ShannonEntropyMeasure
diversity_measure = ShannonEntropyMeasure()

## Set up Language Model

In [12]:
from langdiversity.models import OpenAIModel
from langdiversity.parser import extract_last_letters
TEMPERATURE = 0.8
model = OpenAIModel(openai_api_key=openai_api_key, extractor=extract_last_letters, temperature=TEMPERATURE)

## Set up Diversity Measure Calculator Utility

In [13]:
from langdiversity.utils import DiversityMeasureCollector
diversity_collector = DiversityMeasureCollector(model=model, num_responses=10, diversity_measure=diversity_measure)

## Set up Few-Shot Prompts

In [14]:
original_question = "\nAt the end, say 'the answer is [put the concatenated word here]'.\nQuestion: Take the last letter of each word in \"Tal Evan Lesley Sidney\" and concatenate them.."

fewshot_prompts = [
    f"\nPrompt 1:\nExample 1: Concatenate the last letters of 'Hello World' -> 'od'\nExample 2: Concatenate the last letters of 'Python Ruby Sapphire' -> 'nye'\nExample 3: Concatenate the last letters of 'Learning Teaching Assistant' -> 'ggt'\nExample 4: Concatenate the last letters of 'Reading Writing Center' -> 'ggr'\nExample 5: Concatenate the last letters of 'Singing Dancing Person' -> 'ggn'{original_question}",
    f"\nPrompt 2:\nExample 1: Get the last letters of 'Innovation Creation' and concatenate them -> 'nn'\nExample 2: Get the last letters of 'Motivation Dedications' and concatenate them -> 'ns'\nExample 3: Get the last letters of 'Education Solute' and concatenate them -> 'ne'\nExample 4: Get the last letters of 'Integration Pizza' and concatenate them -> 'na'\nExample 5: Get the last letters of 'Inspiration Aspire' and concatenate them -> 'ne'{original_question}",
    f"\nPrompt 3:\nExample 1: For 'Dance Sing', the last letters when concatenated form -> 'eg'\nExample 2: For 'Jump School', the last letters when concatenated form -> 'pl'\nExample 3: For 'Run Walk', the last letters when concatenated form -> 'nk'\nExample 4: For 'Hop Jam', the last letters when concatenated form -> 'pm'\nExample 5: For 'Sit Stand', the last letters when concatenated form -> 'td'{original_question}",
    f"\nPrompt 4:\nExample 1: From 'Cake Bake', the concatenated last letters are -> 'ee'\nExample 2: From 'Drive Ride', the concatenated last letters are -> 'ee'\nExample 3: From 'Love Live', the concatenated last letters are -> 'ee'\nExample 4: From 'Give Take', the concatenated last letters are -> 'ee'\nExample 5: From 'Make Bake', the concatenated last letters are -> 'ee'{original_question}",
    f"\nPrompt 5:\nExample 1: 'Read Write Dogma Deed' ends in -> 'dead'\nExample 2: 'Play Stay Game Bin' ends in -> 'yyen'\nExample 3: 'Dream Scream Deem Scheme' ends in -> 'mmme'\nExample 4: 'Teach Reach Hands Sky' ends in -> 'hhsy'\nExample 5: 'Learn Earn Pay Gap' ends in -> 'nnyp'{original_question}"
]

## Pass the questions to the Language Model, perform calculation based on the diversity of its responses.

In [15]:
diversity_collector.collect(fewshot_prompts, verbose=True)  

[1/5] Collecting 10 responses... |

[1/5] Collecting 10 responses... ✓
[1/5] Performing diversity measure calculations... ✓
Responses: lnyy, lnyy, lnyy, lnyy, lnyy, lndy, lnyy, lnyy, ylne, lnyy
Diversity Measure (ShannonEntropyMeasure): 0.9219280948873623
[2/5] Collecting 10 responses... ✓
[2/5] Performing diversity measure calculations... ✓
Responses: lnyy, lnyy, lnyy, lne, lnyy, lne, lnyy, lnyy, lnyy, lnyy
Diversity Measure (ShannonEntropyMeasure): 0.7219280948873623
[3/5] Collecting 10 responses... ✓
[3/5] Performing diversity measure calculations... ✓
Responses: lyn, lnyy, lyn, lnyd, leyn, lnyy, lnyy, lnevye, , lney
Diversity Measure (ShannonEntropyMeasure): 2.6464393446710153
[4/5] Collecting 10 responses... ✓
[4/5] Performing diversity measure calculations... ✓
Responses: lne, lndy, lne, lnyy, lnyy, lnyy, lnyy, lly, llny, lnyd
Diversity Measure (ShannonEntropyMeasure): 2.321928094887362
[5/5] Collecting 10 responses... ✓
[5/5] Performing diversity measure calculations... ✓
Responses: lnyy, lney, lnyy, lnyy, lndy, l

## Probability of Failure vs Diversity Measure

In [16]:
import pandas as pd

data = {
    "Prompt": [f"Prompt {i+1}" for i in range(len(diversity_collector.data))],
    "Responses": [item['responses'] for item in diversity_collector.data],
    "Probability of Failure": [
        (len(item['responses']) - item['responses'].count('lnyy')) / len(item['responses']) 
        for item in diversity_collector.data
    ],
    "Diversity Score": [item['diversity'] for item in diversity_collector.data],
    "Temperature": TEMPERATURE
}

df = pd.DataFrame(data)
df


,Prompt,Responses,Probability of Failure,Diversity Score,Temperature
0,Prompt 1,"[lnyy, lnyy, lnyy, lnyy, lnyy, lndy, lnyy, lny...",0.2,0.921928,0.8
1,Prompt 2,"[lnyy, lnyy, lnyy, lne, lnyy, lne, lnyy, lnyy,...",0.2,0.721928,0.8
2,Prompt 3,"[lyn, lnyy, lyn, lnyd, leyn, lnyy, lnyy, lnevy...",0.7,2.646439,0.8
3,Prompt 4,"[lne, lndy, lne, lnyy, lnyy, lnyy, lnyy, lly, ...",0.6,2.321928,0.8
4,Prompt 5,"[lnyy, lney, lnyy, lnyy, lndy, lnds, lnevye, l...",0.7,2.446439,0.8


## Set up Prompt Selection Utility

In [17]:
from langdiversity.utils import PromptSelection
filter = PromptSelection(data=diversity_collector.data, selection="min")
prompts = filter.select()

## Display Prompt with the Minimum Diversity Measure

In [18]:
import re

prompt_results = {
    "Selected Prompt": [re.search(r"Prompt (\d+):", prompt).group(1) for prompt in prompts['selected_prompts']],
    "Diversity Score": [prompts['diversity']] * len(prompts['selected_prompts']),
}

df = pd.DataFrame(prompt_results)
df

,Selected Prompt,Diversity Score
0,2,0.721928
